<a href="https://colab.research.google.com/github/abidshafi/Data-Management-AWS-/blob/main/Assignment3_Fall_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 03 - Due Monday, Nov 22 at 4 pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.

*Deadline*: Monday, November 22, 4 pm. 

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered during a game of the MLB World Series in 2017 featuring the Los Angeles Dodgers and Houston Astros.  This first file tells you a little bit about how it was gathered:

In [ ]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt

--2021-11-20 06:16:15--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.225.240
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.225.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920 (1.9K) [text/plain]
Saving to: ‘9670f3399f774789b7c3e18975d25611-README.txt’

9670f3399f774789b7c 100%[===================>]   1.88K  --.-KB/s    in 0s      

2021-11-20 06:16:15 (98.1 MB/s) - ‘9670f3399f774789b7c3e18975d25611-README.txt’ saved [1920/1920]



In [ ]:
!cat 9670f3399f774789b7c3e18975d25611-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: 9670f3399f774789b7c3e18975d25611
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2017, 11:21:04 p.m. EDT
Deduplicate: No

COLLECTION INFORMATION
Collection name: test set for world series
Collection id: 34e3f7460b5c4df09d64a1e61fd81238
Collection set: mlb-test (collection set id d6e8c27b1bc942e78790aa55a82b3a7a)
Harvest type: Twitter filter
Collection description: running for just one hour, just for fun.

Harvest options:
Media: No
Web resources: No

Seeds:
* Track: dodgers,astros - Active

Change log:

Change to test set for world series (collection) on Oct. 30, 2017, 10:59:56 p.m. EDT by dchud:
* is_active: "True" changed to "False"

Change to test set for world series (collection) on Oct. 30, 2017, 10:58:51 p.m. EDT by dchud:
* is_on: "True" changed to "False"

Change to test set for world series (collection) on Oct. 29, 2017, 8:01:24 p.m. EDT by dch

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 
You should not need to know anything about baseball to complete this assignment.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 
### Q1.1 - Select at least two and obtain them using `wget`.  Verify the file sizes using the command line. (5 points)

Each file should contain exactly 100,000 tweets.  

*Note*: you are only required to use two of these files, but you may use 3 or more.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json #first file

--2021-11-20 06:16:26--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.248.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.248.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  49.0MB/s    in 13s     

2021-11-20 06:16:39 (42.3 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



In [ ]:
# Edit your answer here, adding more cells as needed
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json #second file

--2021-11-20 06:16:41--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.17.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.17.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549995846 (525M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_004.json’

9670f3399f774789b7c 100%[===================>] 524.52M  21.5MB/s    in 15s     

2021-11-20 06:16:55 (36.1 MB/s) - ‘9670f3399f774789b7c3e18975d25611_004.json’ saved [549995846/549995846]



For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [ ]:
!cat *.json | shuf -n 1 > example-tweet.json

In [ ]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "quote_count": 0,
  "contributors": null,
  "truncated": false,
  "text": "RT @HtOwNzOuTLaW713: THE LEGEND OF BREGMAN CONTINUES!!!! ASTROS WIN!!!! 3-2 #Earnhistory https://t.co/2dKFYKbmOy",
  "is_quote_status": false,
  "in_reply_to_status_id": null,
  "reply_count": 0,
  "id": 924877957499424768,
  "favorite_count": 0,
  "entities": {
    "user_mentions": [
      {
        "id": 349940141,
        "indices": [
          3,
          19
        ],
        "id_str": "349940141",
        "screen_name": "HtOwNzOuTLaW713",
        "name": "Choke City \ud83e\udd18\ud83c\udffe"
      }
    ],
    "symbols": [],
    "hashtags": [
      {
        "indices": [
          76,
          88
        ],
        "text": "Earnhistory"
      }
    ],
    "urls": [],
    "media": [
      {
        "source_user_id": 349940141,
        "source_status_id_str": "924873728973996032",
        "expanded_url": "https://twitter.com/HtOwNzOuTLaW713/status/924873728973996032/photo/1",
        "display_url": "pi

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

Find the number of tweets using command line

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
!wc -l 9670f3399f774789b7c3e18975d25611_003.json

100000 9670f3399f774789b7c3e18975d25611_003.json


In [ ]:
# Edit your answer here, adding more cells as needed
!wc -l 9670f3399f774789b7c3e18975d25611_004.json

100000 9670f3399f774789b7c3e18975d25611_004.json


### Q1.2 - Start up Spark, and verify the file sizes. (5 Points)

We will use our normal startup sequence here:

In [ ]:
import os

In [ ]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
from pyspark import SparkContext

In [ ]:
spark = SparkContext(appName='Assignment-03')

21/11/20 06:18:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
spark

<SparkContext master=local[*] appName=Assignment-03>

In [ ]:
from pyspark import SQLContext

In [ ]:
sqlc = SQLContext(spark)

In [ ]:
sqlc

In [ ]:
#tweets = sqlc.read.json("9670f3399f774789b7c3e18975d25611_*.json")
tweets = sqlc.read.json("9670f3399f774789b7c3e18975d25611_*.json")

21/11/20 06:19:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
tweets.count()

200000

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

In [ ]:
tweets

DataFrame[contributors: string, coordinates: struct<coordinates:array<double>,type:string>, created_at: string, display_text_range: array<bigint>, entities: struct<hashtags:array<struct<indices:array<bigint>,text:string>>,media:array<struct<display_url:string,expanded_url:string,id:bigint,id_str:string,indices:array<bigint>,media_url:string,media_url_https:string,sizes:struct<large:struct<h:bigint,resize:string,w:bigint>,medium:struct<h:bigint,resize:string,w:bigint>,small:struct<h:bigint,resize:string,w:bigint>,thumb:struct<h:bigint,resize:string,w:bigint>>,source_status_id:bigint,source_status_id_str:string,source_user_id:bigint,source_user_id_str:string,type:string,url:string>>,symbols:array<struct<indices:array<bigint>,text:string>>,urls:array<struct<display_url:string,expanded_url:string,indices:array<bigint>,url:string>>,user_mentions:array<struct<id:bigint,id_str:string,indices:array<bigint>,name:string,screen_name:string>>>, extended_entities: struct<media:array<struct<display_

To issue SQL queries, we need to register a table based on `tweets`:

In [ ]:
tweets.createOrReplaceTempView("tweets")

In [ ]:
tweets.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user']

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL. (10 Points)

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
tweets.groupBy("lang").count().orderBy('count', ascending = False).show(10)

+----+------+
|lang| count|
+----+------+
|  en|177083|
|  es| 14388|
| und|  6024|
|  in|   492|
|  fr|   384|
|  pt|   303|
|  tl|   263|
|  ht|   173|
|  nl|   156|
|  ja|   141|
+----+------+
only showing top 10 rows



In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
         SELECT lang, COUNT(lang) 
         FROM tweets
         GROUP BY lang
         ORDER BY count(lang) DESC
         """).show(10)

+----+-----------+
|lang|count(lang)|
+----+-----------+
|  en|     177083|
|  es|      14388|
| und|       6024|
|  in|        492|
|  fr|        384|
|  pt|        303|
|  tl|        263|
|  ht|        173|
|  nl|        156|
|  ja|        141|
+----+-----------+
only showing top 10 rows



### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL. (10 Points)

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
# I got user.time_zone from the example json file 
tweets.groupBy('user.time_zone').count().orderBy('count', ascending = False).show(10)

+--------------------+-----+
|           time_zone|count|
+--------------------+-----+
|                null|84004|
|Pacific Time (US ...|36021|
|Central Time (US ...|32514|
|Eastern Time (US ...|17357|
|             Arizona| 5543|
|Mountain Time (US...| 5083|
|Atlantic Time (Ca...| 2177|
|         Mexico City| 1967|
|               Quito| 1777|
|             Caracas| 1760|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
    SELECT user.time_zone, COUNT(*) AS Count 
    FROM tweets
    GROUP BY user.time_zone
    ORDER BY Count DESC 
    """).show(10)

+--------------------+-----+
|           time_zone|Count|
+--------------------+-----+
|                null|84004|
|Pacific Time (US ...|36021|
|Central Time (US ...|32514|
|Eastern Time (US ...|17357|
|             Arizona| 5543|
|Mountain Time (US...| 5083|
|Atlantic Time (Ca...| 2177|
|         Mexico City| 1967|
|               Quito| 1777|
|             Caracas| 1760|
+--------------------+-----+
only showing top 10 rows



### Q2.3 - How many tweets mention the Dodgers?  How many mention the Astros?  How many mention both? (10 Points)

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
         SELECT COUNT(TEXT) AS Mention_Dodgers
         FROM tweets
         WHERE LOWER(tweets.text) LIKE "%dodgers%"
         """).show()

+---------------+
|Mention_Dodgers|
+---------------+
|          80786|
+---------------+



In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
         SELECT COUNT(TEXT) AS Mention_Astros
         FROM tweets
         WHERE LOWER(tweets.text) LIKE "%astros%"
         """).show()

+--------------+
|Mention_Astros|
+--------------+
|        128346|
+--------------+



In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
         SELECT COUNT(TEXT) AS Mention_Both
         FROM tweets
         WHERE (LOWER(tweets.text) LIKE "%dodgers%" 
         AND LOWER(tweets.text) LIKE "%astros%")
         """).show()

+------------+
|Mention_Both|
+------------+
|       27991|
+------------+



#### Edit your answer here, adding more cells as needed
I used SQL to answer the question because : 
1) The first step was to convert all the text to lower case to find all the words, easier for me to do in SQL
2) The same step had to be repeated for dodgers and astros individually
3) To get the tweet with both words, I had to use a simple AND line and the rest of the code format was the same 

## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location (10 Points)

In which users' locations are the Astros and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings.

Hint:  you may use either the time zones or user-specified locations for this question.

**Answer**

In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS Count 
    FROM tweets
    WHERE LOWER(tweets.text) LIKE "%astros%"
    GROUP BY user.location
    ORDER BY COUNT(user.location) DESC 
    """).show(10)

+---------------+-----+
|       location|Count|
+---------------+-----+
|    Houston, TX| 9573|
|     Texas, USA| 2178|
|          Texas| 1406|
|Los Angeles, CA| 1272|
|        Houston| 1162|
| Houston, Texas|  990|
|  United States|  970|
|     Austin, TX|  791|
|California, USA|  718|
|San Antonio, TX|  656|
+---------------+-----+
only showing top 10 rows



In [ ]:
# Edit your answer here, adding more cells as needed
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS Count 
    FROM tweets
    WHERE LOWER(tweets.text) LIKE "%dodgers%"
    GROUP BY user.location
    ORDER BY COUNT(user.location) DESC 
    """).show(10)

+---------------+-----+
|       location|Count|
+---------------+-----+
|Los Angeles, CA| 3228|
|California, USA| 1400|
|    Houston, TX| 1352|
|    Los Angeles|  994|
|  United States|  794|
|     Texas, USA|  500|
|     California|  453|
|         México|  332|
|      Venezuela|  306|
|  Las Vegas, NV|  284|
+---------------+-----+
only showing top 10 rows



#### Edit your answer here, adding more cells as needed
* I expected the tweet count to be highest at the home base for each team, which was the result in this example. Astros has a higher tweet count at their home based Houston. Dodgers has a higher tweet count at their home based Los Angeles. 
* Based on the various location example "Houston, TX" vs "Texas, USA" there seems to be various methods twitter uses to record the tweet location for each tweet. The same applies for Los Angeles. 
* Overall we can see Texas as a whole has more users that mentioned astros in the tweet, which could mean Texas might have a higher fan following for the MLB, making them the most active location.
* It is interestring to see international users from Venezuela and Mexio tweeting about the dodgers.

### Q3.2 - Which Twitter users are being replied to the most? (10 Points)

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT in_reply_to_screen_name, COUNT(in_reply_to_screen_name) AS Count
    FROM tweets
    GROUP BY in_reply_to_screen_name
    ORDER BY Count DESC
    """).show(10)

+-----------------------+-----+
|in_reply_to_screen_name|Count|
+-----------------------+-----+
|                 astros| 1819|
|                Dodgers| 1521|
|                    MLB|  325|
|          stephenasmith|  285|
|              adevaldes|  134|
|        JustinVerlander|  110|
|               MLBONFOX|  106|
|           NancySinatra|   77|
|          rolandsmartin|   70|
|                ABREG_1|   66|
+-----------------------+-----+
only showing top 10 rows



In [ ]:
# Edit your answer here, adding more cells as needed
tweets.groupBy('in_reply_to_screen_name').count().orderBy('count', ascending = False).show(10)

+-----------------------+------+
|in_reply_to_screen_name| count|
+-----------------------+------+
|                   null|181219|
|                 astros|  1819|
|                Dodgers|  1521|
|                    MLB|   325|
|          stephenasmith|   285|
|              adevaldes|   134|
|        JustinVerlander|   110|
|               MLBONFOX|   106|
|           NancySinatra|    77|
|          rolandsmartin|    70|
+-----------------------+------+
only showing top 10 rows



#### Edit your answer here, adding more cells as needed
* I ran SQL and datafame to compare the results and noticed 1 different output, "null".
* I believe the null value is created becuase not every tweet replied to is considered a tweet. They were 181,219 tweets replied to without addressing to anyone based on the dataframe. 
* Ignoring the nulls, the top twitter users being replied to is the teams themselves. Astros and Dodgers respectifully. I think they are on top becuase the team usually post key updates about the team and game. Followers would always want to know information to spread the news or talk about it.

### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers? (10 Points)

Provide both the screen names and follower counts for each.

Discuss your findings.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate
# VERIFIED USERS
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS Count
    FROM tweets
    WHERE user.verified = 'true'
    GROUP BY user.screen_name
    ORDER BY Count DESC
    """).show(10)

+--------------+--------+
|   screen_name|   Count|
+--------------+--------+
|        cnnbrk|53191119|
|       nytimes|39959480|
|           CNN|38209973|
|       Reuters|18937529|
|       FoxNews|16272836|
|           ABC|12551437|
|washingtonpost|11417638|
|   lopezdoriga| 7859821|
|           MLB| 7841255|
|           NPR| 7289492|
+--------------+--------+
only showing top 10 rows



In [ ]:
# Edit this cell and add more as appropriate
# UNVERIFIED USERS
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS Count
    FROM tweets
    WHERE user.verified = 'false'
    GROUP BY user.screen_name
    ORDER BY Count DESC
    """).show(10)

+-------------+------+
|  screen_name| Count|
+-------------+------+
|      chochos|833669|
|   TexasHumor|816301|
|el_carabobeno|725952|
|     PAMsLOvE|712254|
|      jilevin|568341|
|  sun_das_ill|559669|
| DiegoArcos14|544926|
|  TVCDeportes|543094|
|     EP_Mundo|538525|
|       LALATE|516146|
+-------------+------+
only showing top 10 rows



#### Edit this cell and add more as appropriate
* For the top 10 most VERIFIED user followers, which consists majorly of news channels. Nowadays people prefer to read quick headlines vs going through an entire newspaper and I believe this is the reason why news channels are on the top 10. All the verfied users have a far superior follower count compared to the unverified users. Maybe in order to get verfied there is a minimum number of followers needed to get the verfied tag.
* For the UNVERFIED users we can see a huge difference in the follower count compared to verified. It looks like there are individual user account who might be influencers and have a comparably small follower base.


### Q3.4 - What are the most popular sets of hashtags among users with many followers?  Are they the same as among users with few followers? (15 Points)

Decide for yourself exactly how many followers you believe to be "many", and explain your decision.  You may use queries and statistics to support this decision if you wish.

Hint: if your sample tweet above does not include hashtags under the `entities` field, generate a new example by running the `shuf` command again until you find one that does.

Hint 2: the hashtag texts will be in an array, so you may need some functions you haven't used before.  If you're using SQL, see the docs for [Hive SQL](https://docs.treasuredata.com/display/public/PD/Hive+0.13.0+Function+Reference) for details, (and consider `CONCAT_WS`, for example).

Discuss your findings.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate
tweets.describe("user.followers_count").show()

+-------+------------------+
|summary|   followers_count|
+-------+------------------+
|  count|            200000|
|   mean|       5162.377275|
| stddev|202619.93499268303|
|    min|                 0|
|    max|          53191119|
+-------+------------------+



#### Edit this cell and add more as appropriate
Based on the summary of the data we can see that there are users that have min followers in the range of minimum 0 and maximum 53,191,119. The average followers per user is approx 5,162 but we also have to look at the standard devitiation of 202,619 which is extremly high. Hence we can not really consider the mean as the data is highly skewed. Based on an online research it is said than an average user has around 700 followers which is a number very far from our mean. Relating to excercise 3.3 we found the top 10 users with highest followers. Relating to that question, Let's take the top 15% (many followers) and bottom 15% (few followers) from our entire dataset and see the results. This will be a good differentiator between the many and few user

In [ ]:
# Edit this cell and add more as appropriate
tweets.filter("user.followers_count > 1360").count()   #the top 15% (many followers)

30015

In [ ]:
# Edit this cell and add more as appropriate
tweets.filter("user.followers_count <= 81").count()    #the bottom 15% (few followers)

29927

In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT CONCAT_WS(',', entities.hashtags.text) AS Hashtag, COUNT(entities.hashtags.text) AS Count
    FROM tweets
    WHERE user.followers_count > 1360
    GROUP BY (Hashtag)
    ORDER BY(Count) DESC
    """).show(10)

+--------------------+-----+
|             Hashtag|Count|
+--------------------+-----+
|                    |16486|
|         WorldSeries| 2406|
|         EarnHistory| 1627|
|              Astros|  665|
|             Dodgers|  481|
|           ASTROSWIN|  400|
|ThisTeam,WorldSeries|  325|
|ThisTeam,ThisTeam...|  226|
|  Astros,WorldSeries|  200|
|            ThisTeam|  185|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT CONCAT_WS(',', entities.hashtags.text) AS Hashtag, COUNT(entities.hashtags.text) AS Count
    FROM tweets
    WHERE user.followers_count <= 81
    GROUP BY (Hashtag)
    ORDER BY(Count) DESC
    """).show(10)

+--------------------+-----+
|             Hashtag|Count|
+--------------------+-----+
|                    |14432|
|         EarnHistory| 2606|
|         WorldSeries| 1996|
|    LetsCloseThisOut|  984|
|           ASTROSWIN|  701|
|              Astros|  495|
|ThisTeam,ThisTeam...|  461|
|ThisTeam,WorldSeries|  418|
|             Dodgers|  353|
|               HR4HR|  274|
+--------------------+-----+
only showing top 10 rows



#### Edit this cell and add more as appropriate
* Most the hastags were similar for the users with many and few followers. Only hashtag HR4HR with users with few followers seemed to be something relating to another topic.
* For users with MANY followers the only 2 unique hashtags were: "Astros,WorldSeries" AND "ThisTeam"
* For users with FEW followers the only 2 unique hashtags were: "LetsCloseThisOut" AND "HR4HR"

### Q3.5 - Analyze common words in tweet text (15 Points)

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", add a filter that removes at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  To split lines into words, a simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation.

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, you may find words like "thisteam" or "earnhistory". Find out if they are more frequently used by Dodgers or Astros fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate
# The code below will help us determine the top 20 words uses which will help create "stop_words"
tweets.rdd.flatMap(lambda r: r['text'].split(' ')) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(20, key=lambda pair: -pair[1])

[('RT', 120184),
 ('the', 86972),
 ('a', 40586),
 ('to', 40532),
 ('Astros', 39438),
 ('', 37919),
 ('in', 34093),
 ('#ThisTeam', 31176),
 ('Dodgers', 27512),
 ('#WorldSeries', 25982),
 ('@astros:', 25207),
 ('of', 20547),
 ('#EarnHistory', 19880),
 ('I', 19755),
 ('@astros', 19631),
 ('is', 19595),
 ('and', 18987),
 ('The', 17945),
 ('game', 17196),
 ('this', 17187)]

In [ ]:
# Edit this cell and add more as appropriate
stop_words=["a", "an", "the", "RT","this", "that", "these", "those","is", "it", "in","this", "that", "these", 
            "those","if", "or", "of","", "I","i","The","to","and", "for", "THE"]

In [ ]:
# Edit this cell and add more as appropriate
tweets.rdd.flatMap(lambda r: r["text"].split(' ')) \
    .filter(lambda s: s not in stop_words) \
    .map(lambda t: (t,1)) \
    .reduceByKey(lambda a, b: a +b) \
    .takeOrdered(10, key=lambda pair: -pair[1]) 

[('Astros', 39438),
 ('#ThisTeam', 31176),
 ('Dodgers', 27512),
 ('#WorldSeries', 25982),
 ('@astros:', 25207),
 ('#EarnHistory', 19880),
 ('@astros', 19631),
 ('game', 17196),
 ('win', 14614),
 ('@Dodgers', 13769)]

In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT CONCAT_WS(',', entities.hashtags.text) AS Hashtag, COUNT(entities.hashtags.text) AS Count
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text)= "EarnHistory"
    AND LOWER(tweets.text) LIKE "%astros%"
    GROUP BY (Hashtag)
    ORDER BY(Count) DESC
    """).show()

# 17,298 users mentioned Astros and #EarnHistory in their tweet

+-----------+-----+
|    Hashtag|Count|
+-----------+-----+
|EarnHistory|17298|
+-----------+-----+



In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT CONCAT_WS(',', entities.hashtags.text) AS Hashtag, COUNT(entities.hashtags.text) AS Count
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text)= "EarnHistory"
    AND LOWER(tweets.text) LIKE "%dodgers%"
    GROUP BY (Hashtag)
    ORDER BY(Count) DESC
    """).show()

# 281 users mentioned Dodgers and #EarnHistory in their tweet

+-----------+-----+
|    Hashtag|Count|
+-----------+-----+
|EarnHistory|  281|
+-----------+-----+



In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS count
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text)= "EarnHistory"
    GROUP BY user.location
    ORDER BY count DESC
    """).show(10)

# The hashtag #EarnHistory was dominantly used in Texas which was home for the Astros team.

+-------------------+-----+
|           location|count|
+-------------------+-----+
|        Houston, TX| 2267|
|         Texas, USA|  436|
|              Texas|  294|
|            Houston|  227|
|     Houston, Texas|  212|
|         Austin, TX|  162|
|                HTX|  124|
|    San Antonio, TX|  120|
|College Station, TX|  100|
|                htx|   75|
+-------------------+-----+
only showing top 10 rows



In [ ]:
# Edit this cell and add more as appropriate
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS Count
    FROM tweets
    WHERE CONCAT_WS(',', entities.hashtags.text)= "WorldSeries" 
    AND user.verified = 'true'
    GROUP BY user.screen_name
    ORDER BY count DESC
    """).show(10)

# Below is a list of the top 10 verified users that have used the hashtag #WorldSeries in their tweets.

+--------------+--------+
|   screen_name|   Count|
+--------------+--------+
|       FoxNews|16272836|
|           ABC|12551437|
|           MLB| 7840418|
|BleacherReport| 5467676|
|      USATODAY| 3496475|
|    InesSainzG| 3232112|
|          wilw| 3216340|
|AztecaDeportes| 2154308|
|     FOXSports| 1892256|
|       Dodgers| 1828991|
+--------------+--------+
only showing top 10 rows



#### Edit this cell and add more as appropriate

* The word "EarnHistory" was the 6th highest frequency word that was used in tweets amoungst all the users. 17,298 users mentioned Astros and EarnHistory in their tweet. Whereas 281 users mentioned Dodgers and EarnHistory in their tweet. We can say overall that Astros fans were using EarnHistory more frequently.
* The words EarnHistory was mainly uses in Texas which could mean that this word has something related to the Astros team. Maybe the literal meaning is that if they won the game they would break some kind of record and "earnhistory". 
* The word "WorldSeries" was the 4th highest frequency word that was used in tweets amoungst all the users. I wanted to check who the top 10 verified users were that were mentioning the hashtag #WorldSeries in their tweets. It was interesting to see the top count was for Fox News and MLB was 3rd, this could mean that Fox news were reporting news more frequently and had a larger follower base.